# Validation using various techniques

- K fold cross validation

It consists of splitting the available data into K partitions (typically = 4 or 5), instantiating K
identical models, and training each one on (K – 1) partitions while evaluating on the remaining partition.
The validation score for the model used is then the average of the K validation scores obtained.

<img src="../images/Screenshot 2021-12-16 123443.jpg">

In [2]:
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.metrics import mean_absolute_error

### create model

In [ ]:
def create_model(shape):
    # layers of the neural net
    input_layer = InputLayer(input_shape=shape)
    hidden_1 = Dense(units=64, activation=relu)
    hidden_2 = Dense(units=64, activation=relu)
    output_layer = Dense(units=1)

    # compilation
    model = Sequential([input_layer, hidden_1, hidden_2, output_layer])
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.01),
                  loss=mean_squared_error,
                  metrics=[mean_absolute_error])

    return model